In [245]:
import pickle
import pandas as pd
import re
import numpy as np

In [2]:
# 读取
shanghai_new = pickle.load(open( './shanghai_new.pkl', 'rb'))
hangzhou_new = pickle.load(open( './hangzhou_new.pkl', 'rb'))

In [98]:
l = []
for key in shanghai_new.keys():
    l.append(len(shanghai_new[key].keys()))
    
print('爬取房屋的总数：',sum(l))

爬取房屋的总数： 1616


In [90]:
shanghai_new_df = pd.DataFrame(columns = pd.DataFrame(shanghai_new['pg1']).index)
for i in shanghai_new.keys():
    temp = pd.DataFrame(shanghai_new[i]).T
    shanghai_new_df = pd.concat([shanghai_new_df, temp], ignore_index = True)

In [91]:
# 发现dataframe中每个元素都是一个列表（有可能为空）
def extract_0(x):
    """
    提取列表中第一个元素，若列表为空，则返回none
    """
    try:
        return x[0]
    except:
        return None   

In [92]:
df_shanghai = shanghai_new_df.copy()
# dataframe中每一列数据都从列表中提取出来
for col in df_shanghai.columns:
    df_shanghai[col] = df_shanghai[col].apply(extract_0)

In [235]:
# total_price 需要正则表达式进行处理
# area 需要正则表达式进行处理
# unit_price 全都是’元/㎡(均价)‘
df_shanghai.head()

,house_name,total_price,unit_price,unit_price_desc,type,location1,area,total_price_min,total_price_max,area_min,area_max
0,滨江一品苑,总价1600-2597(万/套),117000,元/㎡(均价),住宅,浦东,建面 136-234㎡,1600,2597,136,234
1,红星临港天铂,总价297-330(万/套),33000,元/㎡(均价),住宅,浦东,建面 90-130㎡,297,330,90,130
2,中建·玖里书香,总价273(万/套),28690,元/㎡(均价),住宅,浦东,建面 60-99㎡,273,273,60,99
3,上城院子,总价750-800(万/套),54390,元/㎡(均价),住宅,宝山,建面 101-135㎡,750,800,101,135
4,万科天空之城,总价707(万/套),58600,元/㎡(均价),住宅,青浦,建面 75-160㎡,707,707,75,160


对总价进行处理

In [119]:
temp = df_shanghai.total_price.str.extract('总价([\w-]+)')

In [182]:
# # 观察发现总价中无法提取数字的，都是没有爬取到总价的（即 总价 为空）
# df_shanghai.loc[temp.isna()[0], 'total_price'].value_counts()

In [200]:
# 获取总价的最大、最小值：
# 有区间的正常取最大、最小值；没有区间的最大最小值都直接取数字
df_shanghai['total_price_min'] = temp[0].str.extract(r'([\d]+)-')
df_shanghai['total_price_max'] = temp[0].str.extract(r'-([\d]+)')
df_shanghai.loc[df_shanghai['total_price_min'].isna(), 'total_price_min'] = temp.loc[df_shanghai['total_price_min'].isna(), 0] 
df_shanghai.loc[df_shanghai['total_price_max'].isna(), 'total_price_max'] = temp.loc[df_shanghai['total_price_max'].isna(), 0] 

对面积进行处理

In [229]:
temp = df_shanghai.area.str.extract(r'([\d-]+)')

In [231]:
# 提取为空都是因为原始的爬取数据就是空的
# df_shanghai.loc[df_shanghai.area.str.extract(r'([\d-]+)').isna()[0],'area'].unique()

In [234]:
df_shanghai['area_min'] = temp[0].str.extract(r'([\d]+)-')
df_shanghai['area_max'] = temp[0].str.extract(r'-([\d]+)')
df_shanghai.loc[df_shanghai['area_min'].isna(), 'area_min'] = temp.loc[df_shanghai['area_min'].isna(), 0] 
df_shanghai.loc[df_shanghai['area_max'].isna(), 'area_max'] = temp.loc[df_shanghai['area_max'].isna(), 0] 

单价、总价、面积相关列转换为数值类型

In [247]:
# 发现单价里面有价格待定,将其转换为空值
df_shanghai['unit_price'] = df_shanghai.unit_price.replace('价格待定', np.nan)

In [248]:
col_list = [ 'unit_price', 'area_min', 'area_max', 'total_price_min', 'total_price_max']

In [249]:
df_shanghai[col_list] = df_shanghai[col_list].astype('float')

In [251]:
df_shanghai.to_csv('sh_new_clean.csv')